In [53]:
import requests
import json
import pandas as pd
from config import api_key
from config import token 
import csv

In [54]:
beer_csv = pd.read_csv("addresses2.csv", delimiter=",")
beer_csv_df=pd.DataFrame(beer_csv)
beer_csv_df.head()

,Name,Address,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,City
0,10 Barrel Brewing Co.,1501 E St,San Diego,CA 92101,NaN,NaN,San Diego
1,2Kids Brewing Co.,8680 Miralani Drive #123,San Diego,CA 92126,NaN,NaN,San Diego
2,13 Point Brewing Company,8035 Broadway,Lemon Grove,CA 91945,NaN,NaN,Lemon Grove
3,32 North Brewing Co.,8655 Production Avenue Suite A,San Diego,CA 92121,NaN,NaN,San Diego
4,Abnormal Beer Company,16990 Via Tazon Suite 124,San Diego,CA 92127,NaN,NaN,San Diego


In [55]:
biz_name = beer_csv_df["Name"]
addr = beer_csv_df["Address"]
city = beer_csv_df["City"]
addy = addr.values.tolist()
biz = biz_name.values.tolist()
cities = city.values.tolist()
len(cities)

152

In [56]:
# for index, row in beer_csv_df.iterrows():
#     biz, addy = row['Name'], row['Address']
#     print(a)

In [60]:
ids = []
name = []
address = []
lat = []
lng = []
rating = []
review = []
x = -1
addy 
biz

skippedb = []
skippeda = []
for b, a, c in zip(biz, addy, cities):
    x += 1
    print(f"printing {b} || {x} of {len(biz)} ")
# for b, a, in beer_csv_df.iterrows():
    headers = {'Authorization': 'Bearer %s' % api_key}
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/matches"
    try:
        response = requests.get(url, headers=headers, params=params)
        response_json = response.json()
        idz = response_json["businesses"][0]["id"]
        addresses = response_json["businesses"][0]["location"]["address1"]
        lats = response_json["businesses"][0]["coordinates"]["latitude"]
        lngs = response_json["businesses"][0]["coordinates"]["longitude"]
        names = response_json["businesses"][0]["name"]
        name.append(names)
        address.append(addresses)
        lat.append(lats)
        lng.append(lngs)
        ids.append(idz)
    except:
        skippedb.append(b)
        skippeda.append(a)
        print('Skipping:', b)
        continue

for b, a, c, i, in zip(biz, addy, cities, ids):
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/"
    response = requests.get(url + i, headers=headers, params=params)
    response_json = response.json()
    ratings = response_json["rating"]
    reviews = response_json["review_count"]
    review.append(reviews)
    rating.append(ratings)


printing 10 Barrel Brewing Co. || 0 of 152 
printing 2Kids Brewing Co. || 1 of 152 
printing 13 Point Brewing Company || 2 of 152 
printing 32 North Brewing Co. || 3 of 152 
printing Abnormal Beer Company || 4 of 152 
printing AleSmith Brewing Company || 5 of 152 
printing Align Brewing Co. || 6 of 152 
printing Alpine Beer Company || 7 of 152 
printing Amplified Ale Works || 8 of 152 
printing Amplified Ale Works || 9 of 152 
printing Arcana Brewing Co. || 10 of 152 
printing Attitude Brewing || 11 of 152 
printing Automatic Brewing Company || 12 of 152 
printing Aztec Brewing Company || 13 of 152 
printing Back Street Brewery || 14 of 152 
printing Bagby Beer Comapny || 15 of 152 
printing Ballast Point Brewing & Spirits - Home Brew Mart || 16 of 152 
printing Ballast Point Brewing & Spirits - Little Italy || 17 of 152 
printing Ballast Point Brewing & Spirits - Miramar || 18 of 152 
printing Ballast Point Brewing & Spirits - Scripps Ranch || 19 of 152 
printing Barn Brewery || 20 of

In [61]:
review

[364,
 86,
 33,
 170,
 6,
 405,
 17,
 492,
 857,
 55,
 94,
 96,
 6,
 133,
 297,
 1104,
 447,
 1759,
 1823,
 514,
 173,
 108,
 91,
 54,
 127,
 34,
 175,
 420,
 70,
 650,
 110,
 88,
 134,
 150,
 230,
 241,
 6,
 556,
 153,
 104,
 31,
 76,
 50,
 1206,
 285,
 30,
 324,
 86,
 70,
 77,
 90,
 6,
 232,
 49,
 17,
 98,
 44,
 258,
 139,
 1023,
 994,
 120,
 26,
 20,
 612,
 44,
 186,
 131,
 441,
 6,
 142,
 277,
 100,
 137,
 100,
 797,
 1352,
 1063,
 816,
 1317,
 21,
 190,
 539,
 33,
 168,
 136,
 143,
 201,
 152,
 2,
 45,
 90,
 513,
 163,
 680,
 604,
 103,
 140,
 277,
 52,
 217,
 305,
 112,
 17,
 1359,
 87,
 30,
 64,
 543,
 1915,
 1238,
 1378,
 547,
 194,
 483,
 550,
 80,
 623,
 187,
 100,
 180,
 878,
 200,
 75,
 535,
 350,
 35,
 224,
 38,
 601,
 210,
 35,
 188,
 162,
 2010,
 107,
 121,
 348,
 46,
 67,
 55,
 406,
 159,
 184,
 157]

In [45]:
a = -1
rating = []
review = []
test = []
for x, c in zip(biz, cities):
    a += 1
    params = {'term':x, 'city':c, "location":"CA"}
    url = "https://api.yelp.com/v3/businesses/search"
    response = requests.get(url, headers=headers, params=params)
    response_json = response.json()
#     cit = response_json["businesses"][0]["location"]["city"]
#     test.append(cit)

#     review.append(reviews)
#     rating.append(ratings)
# review

    response_json